In [58]:
from scipy import stats
import math
# Calculate percentage MSE reduction for each dataset and horizon
def Calculate_percentage_MSE_reduction(data):
    results = {}
    for dataset, values in data.items():
        mse_reductions = {}
        for horizon_idx, horizon in enumerate(values['horizons']):
            # Extract MSE values for this horizon across all models
            mse_values = {model: mse[horizon_idx] for model, mse in values['models'].items()}
            # Calculate closest MSE (minimum) excluding NFT
            closest_mse = min(mse for model, mse in mse_values.items() if model != 'nft')
            # Calculate MSE reduction for NFT
            nft_mse = mse_values['nft']
            mse_reduction = (1 - (nft_mse / closest_mse)) * 100
            mse_reductions[horizon] = mse_reduction
        results[dataset] = mse_reductions

    # Print results
    # for dataset, reductions in results.items():
    #     print(f"Dataset: {dataset}")
    #     for horizon, reduction in reductions.items():
    #         print(f"  Horizon {horizon}: {reduction:.2f}% MSE reduction")
    #     print()
    return results

# Calculate the average percentage MSE reduction across horizons for each dataset
def Calculate_average_percentage_MSE(results):
    average_results = {}
    for dataset, values in results.items():
        average_reduction = sum(values.values()) / len(values)
        average_results[dataset] = average_reduction
    print(average_results)
    return average_results

def Calculate_percentage_MSE_reduction_ablation(data_ablation):
    results_ablation_lstm = {}
    for dataset, values in data_ablation.items():
        mse_reductions = {}
        for horizon_idx, horizon in enumerate(values['horizons']):
            # Extract MSE values for this horizon across all models
            mse_values = {model: mse[horizon_idx] for model, mse in values['models'].items()}
            # Calculate closest MSE (minimum) excluding NFT
            closest_mse = min(mse for model, mse in mse_values.items() if model != 'nft' and model != 'fc nft')
            # Calculate MSE reduction for NFT
            nft_mse = mse_values['nft']
            mse_reduction = (1 - (nft_mse / closest_mse)) * 100
            mse_reductions[horizon] = mse_reduction
        results_ablation_lstm[dataset] = mse_reductions
        
    results_ablation_fc = {}
    for dataset, values in data_ablation.items():
        mse_reductions = {}
        for horizon_idx, horizon in enumerate(values['horizons']):
            # Extract MSE values for this horizon across all models
            mse_values = {model: mse[horizon_idx] for model, mse in values['models'].items()}
            # Calculate closest MSE (minimum) excluding NFT
            closest_mse = min(mse for model, mse in mse_values.items() if model != 'nft' and model != 'lstm nft')
            # Calculate MSE reduction for NFT
            nft_mse = mse_values['nft']
            mse_reduction = (1 - (nft_mse / closest_mse)) * 100
            mse_reductions[horizon] = mse_reduction
        results_ablation_fc[dataset] = mse_reductions
        
    
    return results_ablation_lstm, results_ablation_fc

# def average_dict_values(data):
#     if not data:
#         return 0  # Return 0 for an empty dictionary
#     total = sum(data.values())
#     count = len(data)
#     return total / count

def average_dict_values(data):
    if not data:
        return 0  # Return 0 for an empty dictionary
    
    total = 0
    count = 0
    
    for value in data.values():
        if not math.isnan(value):
            total += value
            count += 1
    if count == 0:
        return 0  # Avoid division by zero if all values are NaN
    return total / count

def Calulate_t_test(data):
    t_stat_dict = {}
    p_value_dict = {}

    for dataset, values in data.items():
        t_stat_dict[dataset] = {}
        p_value_dict[dataset] = {}
        nft_vals=values['models']['nft']
        for model, vals in values['models'].items():
            if model != 'nft':
                t_stat, p_value = stats.ttest_rel(nft_vals, vals)
                t_stat_dict[dataset][model] = t_stat
                p_value_dict[dataset][model] = p_value
    return t_stat_dict, p_value_dict

def calculate_average_t_p(t_stat_dict, p_value_dict):
    avg_t_stat = {}
    avg_p_value = {}

    # Calculate average for each dataset
    for dataset in t_stat_dict:
        avg_t_stat[dataset] = sum(t_stat_dict[dataset].values()) / len(t_stat_dict[dataset])
        avg_p_value[dataset] = sum(p_value_dict[dataset].values()) / len(p_value_dict[dataset])

    # Calculate overall average across all datasets
    total_t_stat = sum([sum(vals.values()) for vals in t_stat_dict.values()])
    total_t_count = sum([len(vals) for vals in t_stat_dict.values()])
    overall_avg_t_stat = average_dict_values(avg_t_stat)#total_t_stat / total_t_count

    total_p_value = sum([sum(vals.values()) for vals in p_value_dict.values()])
    total_p_count = sum([len(vals) for vals in p_value_dict.values()])
    overall_avg_p_value = average_dict_values(avg_p_value)#total_p_value / total_p_count

    return avg_t_stat, avg_p_value, overall_avg_t_stat, overall_avg_p_value


In [23]:
mse_data = {
'Traffic': {
    'horizons': [1, 16, 32, 48],
    'models': {
        'nft': [0.09, 0.18, 0.26, 0.4],
        'timesnet': [0.53, 0.58, 0.62, 0.62],
        'patchtst': [0.26, 0.58, 0.63, 0.66],
        'dlinear': [0.4, 0.63, 0.67, 0.72],
        'Autoformer': [0.5, 0.56, 0.67, 0.65],
        'tcn': [0.1, 0.46, 0.49, 0.51],
        'nbeats': [0.2, 0.51, 0.59, 0.71]
    }
},

    'Weather': {
        'horizons': [1, 7, 15, 30, 60, 90, 120, 180],
        'models': {
            'nft': [0.19, 0.24, 0.24, 0.25, 0.26, 0.27, 0.28, 0.27],
            'timesnet': [0.33, 0.35, 0.44, 0.36, 0.37, 0.37, 0.38, 0.37],
            'patchtst': [0.34, 0.36, 0.37, 0.38, 0.40, 0.42, 0.42, 0.43],
            'dlinear': [0.29, 0.32, 0.33, 0.35, 0.37, 0.40, 0.41, 0.40],
            'Autoformer': [0.37, 0.37, 0.37, 0.38, 0.42, 0.44, 0.46, 0.47],
            'tcn': [0.25, 0.29, 0.30, 0.32, 0.36, 0.33, 0.33, 0.35],
            'nbeats': [0.31, 0.33, 0.35, 0.38, 0.40, 0.42, 0.43, 0.44]
        }
    },

    'Electricity': {
        'horizons': [1, 16, 32],
        'models': {
            'nft': [0.04, 0.08, 0.13],
            'timesnet': [0.14, 0.16, 0.17],
            'patchtst': [0.09, 0.20, 0.17],
            'dlinear': [0.07, 0.16, 0.18],
            'Autoformer': [0.20, 0.40, 0.60],
            'tcn': [0.05, 0.32, 0.44],
            'nbeats': [0.06, 0.13, 0.18]
        }
    },

    'Exchange Rate all': {
    'horizons': [1, 16, 32, 48, 96, 192, 336, 720],
    'models': {
        'nft': [0.0001, 0.01, 0.02, 0.02, 0.02, 0.03, 0.04, 0.05],
        'timesnet': [0.05, 0.18, 0.15, 0.11, 0.107, 0.226, 0.367, 0.964],
        'patchtst': [0.01, 0.02, 0.03, 0.04, 0.09, 0.18, 0.4, 0.94],
        'dlinear': [0.01, 0.02, 0.03, 0.05, 0.088, 0.176, 0.313, 0.839],
        'Autoformer': [0.04, 0.04, 0.06, 0.09, 0.197, 0.300, 0.509, 1.447],
        'tcn': [0.2, 0.11, 0.04, 0.06, 0.09, 0.17, 0.24, 0.41],
        'nbeats': [0.01, 0.03, 0.05, 0.12, 0.17, 0.19, 0.21, 0.25]
    }
    },
    
    'ILI': {
        'horizons': [1, 12, 24, 36, 48, 60],
        'models': {
            'nft': [0.05, 0.18, 0.25, 0.31, 0.35, 0.42],
            'timesnet': [0.64, 1.01, 2.317, 1.972, 2.238, 2.027],
            'patchtst': [0.24, 1.25, 1.319, 1.579, 1.553, 1.470],
            'dlinear': [2.32, 2.83, 2.215, 1.963, 2.13, 2.368],
            'Autoformer': [0.84, 2.47, 3.483, 3.103, 2.669, 2.770],
            'tcn': [0.86, 0.98, 0.95, 0.94, 1.01, 1.03],
            'nbeats': [0.3, 1.33, 1.19, 1.44, 1.09, 1.48]
        }
    },
    'Air Quality': {
        'horizons': [1, 5, 10, 15, 20, 25, 30],
        'models': {
            'nft': [0.13, 0.21, 0.28, 0.31, 0.35, 0.38, 0.4],
            'timesnet': [0.74, 0.81, 0.85, 0.87, 0.85, 0.92, 0.96],
            'patchtst': [0.22, 0.58, 0.68, 0.72, 0.75, 0.78, 0.83],
            'dlinear': [0.39, 0.69, 0.78, 0.83, 0.87, 0.93, 1.01],
            'Autoformer': [0.72, 0.86, 0.97, 1.02, 0.98, 1.03, 0.99],
            'tcn': [0.21, 0.42, 0.53, 0.64, 0.69, 0.68, 0.69],
            'nbeats': [0.38, 0.64, 0.82, 0.86, 0.93, 1, 1.1]
        }
    },
    'ECG (600)': {
        'horizons': [1, 5, 10, 15, 20, 25, 30],
        'models': {
            'nft': [0.02, 0.08, 0.07, 0.1, 0.15, 0.15, 0.1],
            'timesnet': [0.02, 0.08, 0.08, 0.1, 0.15, 0.15, 0.17],
            'patchtst': [0.02 ,0.05 ,0.08 ,0.1 ,0.15 ,0.15 ,0.18],
            'dlinear': [0.02 ,0.06 ,0.08 ,0.11 ,0.14 ,0.16 ,0.19],
            'Autoformer': [0.8,0.49,0.38, 0.41 ,0.46 ,0.44 ,0.45],
            'tcn': [ 0.03 ,0.07 ,0.1 ,0.12 ,0.16 ,0.18 ,0.21],
            'nbeats': [ 0.07 ,0.07 ,0.11 ,0.15 ,0.18 ,0.22 ,0.28]
        }
    },
    'ECG': {
        'horizons': [1, 10, 25, 50, 100],
        'models': {
            'nft': [0.04, 0.08, 0.12, 0.14, 0.18],
            'timesnet': [0.31 ,0.49 ,0.625 ,1.16 ,1.09],
            'patchtst': [0.34 ,0.57 ,0.77 ,0.94 ,0.98],
            'dlinear': [0.35 ,0.46 ,0.58 ,0.66 ,0.69],
            'Autoformer': [1.62 ,1.13 ,1.5 ,1.15 ,1.26],
            'tcn': [0.16 ,0.42 ,0.56 ,0.70 ,0.81],
            'nbeats': [0.22 ,0.44 ,0.50 ,0.72 ,0.67]
        }
    },
    'EEG': {
        'horizons': [1, 10, 25],
        'models': {
            'nft': [0.03, 0.1, 0.16],
            'timesnet': [0.16, 0.35, 0.44],
            'patchtst': [0.15, 0.345, 0.42],
            'dlinear': [0.135, 0.365, 0.435],
            'Autoformer': [0.63, 0.545, 0.62],
            'tcn': [0.09, 0.32, 0.39],
            'nbeats': [0.37, 0.335, 0.336]
        }
    },
    'Chorales': {
        'horizons': [1, 2, 3, 4, 5],
        'models': {
            'nft': [0.15, 0.18, 0.20, 0.23, 0.22],
            'timesnet': [0.29, 0.3, 0.31, 0.31, 0.32],
            'patchtst': [0.34, 0.27, 0.29, 0.30, 0.31],
            'dlinear': [0.29, 0.29, 0.40, 0.37, 0.36],
            'Autoformer': [0.30, 0.25, 0.25, 0.26, 0.24],
            'tcn': [0.27, 0.63, 0.62, 0.37, 0.56],
            'nbeats': [0.35, 0.27, 0.27, 0.26, 0.29],
        }
    },

    'ETTh1': {
        'horizons': [1, 10, 24, 48, 96, 192, 336, 720],
        'models': {
            'nft': [0.07, 0.14, 0.18, 0.25, 0.39, 0.43, 0.44, 0.46],
            'timesnet': [0.15, 0.29, 0.33, 0.37, 0.38, 0.44, 0.49, 0.52],
            'patchtst': [0.1, 0.25, 0.29, 0.34, 0.37, 0.41, 0.42, 0.45],
            'dlinear': [0.12, 0.28, 0.31, 0.35, 0.37, 0.41, 0.44, 0.47],
            'Autoformer': [0.17, 0.4, 0.4, 0.427, 0.44, 0.46, 0.49, 0.52],
            'tcn': [0.14, 0.29, 0.43, 0.46, 0.46, 0.54, 0.53, 0.58],
            'nbeats': [0.11, 0.29, 0.33, 0.47, 0.71, 0.78, 0.79, 0.84]
        }
    },
    'ETTh1_long': {
        'horizons': [96, 192, 336, 720],
        'models': {
            'nft': [0.39, 0.43, 0.44, 0.46],
            'timesnet': [0.4, 0.44, 0.49, 0.52],
            'patchtst': [0.37, 0.41, 0.42, 0.45],
            'dlinear': [0.37, 0.41, 0.44, 0.47],
            'Autoformer': [0.43, 0.46, 0.49, 0.52],
            'tcn': [0.46, 0.54, 0.53, 0.58],
            'nbeats': [0.71, 0.78, 0.79, 0.84]
        }
    },
    'ETTh2': {
        'horizons': [1, 10, 24, 48, 96, 192, 336, 720],
        'models': {
            'nft': [0.06, 0.1, 0.13, 0.16, 0.19, 0.23, 0.26, 0.34],
            'timesnet': [0.1, 0.15, 0.2, 0.25, 0.34, 0.4, 0.45, 0.46],
            'patchtst': [0.06, 0.12, 0.17, 0.22, 0.274, 0.34, 0.33, 0.38],
            'dlinear': [0.11, 0.13, 0.18, 0.25, 0.289, 0.38, 0.45, 0.61],
            'Autoformer': [0.12, 0.21, 0.26, 0.32, 0.332, 0.43, 0.48, 0.45],
            'tcn': [0.06, 0.11, 0.15, 0.19, 0.24, 0.48, 0.54, 0.55],
            'nbeats': [0.14, 0.18, 0.24, 0.41, 1.65, 1.95, 1.75, 1.73]
        }
    },
    'ETTm1': {
        'horizons': [1, 10, 24, 48, 96, 192, 336, 720],
        'models': {
            'nft': [0.04, 0.07, 0.11, 0.14, 0.16, 0.24, 0.37, 0.43],
            'timesnet': [0.05, 0.13, 0.20, 0.29, 0.34, 0.37, 0.41, 0.48],
            'patchtst': [0.05, 0.12, 0.21, 0.28, 0.293, 0.33, 0.37, 0.42],
            'dlinear': [0.05, 0.14, 0.24, 0.31, 0.299, 0.34, 0.37, 0.43],
            'Autoformer': [0.09, 0.24, 0.4, 0.56, 0.51, 0.51, 0.51, 0.53],
            'tcn': [0.04, 0.09, 0.14, 0.19, 0.18, 0.47, 0.45, 0.54],
            'nbeats': [0.05, 0.13, 0.23, 0.3, 0.35, 0.5, 0.58, 0.79]
        }
    },
    'ETTm2': {
        'horizons': [1, 10, 24, 48, 96, 192, 336, 720],
        'models': {
            'nft': [0.03, 0.05, 0.07, 0.1, 0.13, 0.17, 0.22, 0.26],
            'timesnet': [0.04, 0.07, 0.11, 0.15, 0.16, 0.25, 0.32, 0.41],
            'patchtst': [0.03, 0.07, 0.1, 0.14, 0.166, 0.22, 0.27, 0.36],
            'dlinear': [0.03, 0.07, 0.11, 0.15, 0.167, 0.22, 0.28, 0.40],
            'Autoformer': [0.07, 0.1, 0.14, 0.165, 0.205, 0.28, 0.34, 0.41],
            'tcn': [0.03, 0.06, 0.09, 0.12, 0.16, 0.3, 0.34, 0.42],
            'nbeats': [0.04, 0.07, 0.14, 0.18, 0.26, 0.59, 0.81, 1.00]
        }
    }
    }

data_nbeats_mase = {
    'Weather': {
        'horizons': [1, 7, 15, 30, 60, 90, 120, 180],
        'models': {
            'nft': [79.68 , 90.39 , 91.27 , 92.2 , 95.39, 95.53 , 96.86 , 97.38],
            'nbeats': [92.77 ,96.86 ,99.92 ,97.02 ,99.48 ,99.89 ,100.69 , 102.27]
        }
    },
    'Electricity': {
        'horizons': [1, 16, 32],
        'models': {
            'nft': [34.06 , 52.14 , 63.29],
            'nbeats': [35.36 , 48.86 ,57.62]
        }
    },
    'Exchange Rate': {
        'horizons': [1, 16, 48],
        'models': {
            'nft': [7.25, 18.71, 30.5],
            'nbeats': [8.2, 17.9, 29.54],
        }
    },
    'ILI': {
        'horizons': [1, 12, 60],
        'models': {
            'nft': [44.86, 85.97, 92.21],
            'nbeats': [49.49, 86.61, 111.63],
        }
    },
    'Air Quality': {
        'horizons': [1, 5, 10, 15, 20, 25, 30],
        'models': {
            'nft': [58.38, 79.23 , 87.01 , 93.56 , 95.79 , 99.38 , 102.58],
             'nbeats': [75.37 ,83.85 ,100.55 ,102.78 ,103.91 ,109.81 ,111.67],
        }
    },
    'ECG (600)': {
        'horizons': [1, 5, 10, 15, 20, 25, 30],
        'models': {
            'nft': [24.07 ,43.86 ,45.26 ,53.87 ,59.81 ,60.03 , 68.3],
            'nbeats': [25.31 ,41.26 ,49.96 ,56.41 ,63.83 ,68.31 , 74.1],
        }
    },
    'ECG': {
        'horizons': [1, 10, 25, 50, 100],
        'models': {
            'nft': [40.47, 64.81, 74.70, 82.94, 88.75],
            'nbeats': [73.5,87.02,96.57,108.48,106.95],
        }
    },
    'EEG': {
        'horizons': [1, 10, 25],
        'models': {
            'nft': [45.83 , 62.90 , 72.23],
            'nbeats': [67.38,82.965,94],
        }
    },
    'Chorales': {
        'horizons': [1, 2, 3, 4, 5],
        'models': {
            'nft': [69.37 , 71.21 , 73.56 , 73.46 , 75.82],
            'nbeats': [70.36 ,71.43 ,71.55 ,72.68 ,76.4],
        }
    }
}

data_ablation_mase = {
    'Weather': {
        'horizons': [1, 7, 15, 30, 60, 90, 120, 180],
        'models': {
            'nft': [0.65, 0.75, 0.76, 0.79, 0.81, 0.82, 0.83, 0.85],
            'lstm nft': [0.73, 0.83, 0.87, 0.88, 0.90, 0.91, 0.91, 0.92],
            'fc nft': [0.86, 0.89, 0.90, 0.90, 0.92, 0.92, 0.91, 0.91]
        }
    },
    'Air Quality': {
        'horizons': [1, 5, 10, 15, 20, 25, 30],
        'models': {
            'nft': [0.15, 0.25, 0.30, 0.34, 0.35, 0.41, 0.39],
            'lstm nft': [0.25, 0.50, 0.52, 0.51, 0.61, 0.65, 0.62],
            'fc nft': [0.30, 0.45, 0.51, 0.56, 0.59, 0.60, 0.63]
        }
    },
    'ECG (600)': {
        'horizons': [1, 5, 10, 15, 20, 25, 30],
        'models': {
            'nft': [0.11, 0.24, 0.25, 0.32, 0.36, 0.37, 0.44],
            'lstm nft': [0.11, 0.20, 0.25, 0.30, 0.35, 0.38, 0.41],
            'fc nft': [0.16, 0.23, 0.27, 0.35, 0.36, 0.41, 0.44]
        }
    },
    'ECG': {
        'horizons': [1, 10, 25, 50, 100],
        'models': {
            'nft': [0.18, 0.34, 0.42, 0.49, 0.54],
            'lstm nft': [0.27, 0.59, 0.613, 0.6825, 0.715],
            'fc nft': [0.47, 0.557, 0.56, 0.6275, 0.6425]
        }
    },
    'EEG': {
        'horizons': [1, 10, 25],
        'models': {
            'nft': [0.14, 0.27, 0.34],
            'lstm nft': [0.185, 0.48, 0.63],
            'fc nft': [0.245, 0.49, 0.585]
        }
    },
    'Chorales': {
        'horizons': [1, 2, 3, 4, 5],
        'models': {
            'nft': [0.27, 0.24, 0.25, 0.27, 0.28],
            'lstm nft': [0.21, 0.22, 0.23, 0.23, 0.24],
            'fc nft': [0.32, 0.27, 0.26, 0.29, 0.32]
        }
    }
}

data_ablation_smape = {
    'Weather': {
        'horizons': [1, 7, 15, 30, 60, 90, 120, 180],
        'models': {
            'nft': [79.68, 90.39, 91.27, 92.27, 95.39, 95.53, 96.86, 97.38],
            'lstm nft': [84.94, 92.57, 93.35, 94.36, 95.72, 96.18, 97.90, 98.64],
            'fc nft': [91.08, 93.95, 95.89, 94.80, 97.08, 98.38, 98.54, 98.12]
        }
    },
    'Air Quality': {
        'horizons': [1, 5, 10, 15, 20, 25, 30],
        'models': {
            'nft': [58.38, 79.23, 87.01, 93.56, 95.79, 99.38, 102.58],
            'lstm nft': [76.03, 113.43, 117.59, 113.79, 126.13, 127.83, 124.97],
            'fc nft': [86.05, 104.42, 113.86, 119.42, 124.11, 125.50, 126.63]
        }
    },
    'ECG (600)': {
        'horizons': [1, 5, 10, 15, 20, 25, 30],
        'models': {
            'nft': [24.07, 43.86, 45.26, 53.87, 59.81, 60.03, 68.39],
            'lstm nft': [25.86, 39.42, 46.68, 52.26, 59.88, 63.40, 66.84],
            'fc nft': [35.07, 43.96, 49.07, 60.21, 59.71, 65.59, 69.99]
        }
    },
    'ECG': {
        'horizons': [1, 10, 25, 50, 100],
        'models': {
            'nft': [40.47, 64.81, 74.70, 82.94, 88.75],
            'lstm nft': [52.91, 87.48, 94.17, 98.95, 103.40],
            'fc nft': [79.23, 86.99, 90.93, 95.28, 98.03]
        }
    },
    'EEG': {
        'horizons': [1, 10, 25],
        'models': {
            'nft': [45.83, 62.90, 72.23],
            'lstm nft': [49.80, 88.16, 104.18],
            'fc nft': [59.50, 89.85, 104.16]
        }
    },
    'Chorales': {
        'horizons': [1, 2, 3, 4, 5],
        'models': {
            'nft': [69.37, 71.21, 73.56, 73.46, 75.82],
            'lstm nft': [95.14, 90.37, 86.56, 88.99, 91.87],
            'fc nft': [86.47, 76.74, 75.65, 79.05, 84.91]
        }
    }
}

data_ablation_mse = {
    'Weather': {
        'horizons': [1, 7, 15, 30, 60, 90, 120, 180],
        'models': {
            'nft': [0.19, 0.24, 0.24, 0.25, 0.26, 0.27, 0.28, 0.27],
            'lstm nft': [0.29, 0.33, 0.36, 0.37, 0.39, 0.41, 0.42, 0.44],
            'fc nft': [0.41, 0.39, 0.40, 0.39, 0.40, 0.41, 0.42, 0.41]
        }
    },
    'Air Quality': {
        'horizons': [1, 5, 10, 15, 20, 25, 30],
        'models': {
            'nft': [0.13, 0.21, 0.28, 0.31, 0.35, 0.38, 0.40],
            'lstm nft': [0.38, 1.21, 1.29, 1.25, 1.76, 2.02, 1.78],
            'fc nft': [0.49, 1.04, 1.27, 1.51, 1.62, 1.72, 1.88]
        }
    },
    'ECG (600)': {
        'horizons': [1, 5, 10, 15, 20, 25, 30],
        'models': {
            'nft': [0.02, 0.08, 0.07, 0.10, 0.15, 0.15, 0.10],
            'lstm nft': [0.07, 0.07, 0.10, 0.14, 0.15, 0.18, 0.21],
            'fc nft': [0.08, 0.08, 0.12, 0.16, 0.19, 0.22, 0.24]
        }
    },
    'ECG': {
        'horizons': [1, 10, 25, 50, 100],
        'models': {
            'nft': [0.04, 0.08, 0.12, 0.14, 0.18],
            'lstm nft': [0.085, 0.3675, 0.4375, 0.5225, 0.5825],
            'fc nft': [0.225, 0.3375, 0.4025, 0.475, 0.5175]
        }
    },
    'EEG': {
        'horizons': [1, 10, 25],
        'models': {
            'nft': [0.03, 0.10, 0.16],
            'lstm nft': [0.10, 0.445, 0.655],
            'fc nft': [0.125, 0.425, 0.545]
        }
    },
    'Chorales': {
        'horizons': [1, 2, 3, 4, 5],
        'models': {
            'nft': [0.15, 0.18, 0.20, 0.23, 0.22],
            'lstm nft': [0.36, 0.30, 0.26, 0.245, 0.23],
            'fc nft': [0.35, 0.29, 0.28, 0.29, 0.28]
        }
    }
}


In [22]:
import pandas as pd

datasets = []
horizons_list = []
predictive_ranges = []
nft_mses = []
closest_baseline_mses = []
improvements = []

for dataset in mse_data:
    horizons = mse_data[dataset]['horizons']
    nft_mse = mse_data[dataset]['models']['nft']
    
    # Get the MSE values of all baseline models excluding NFT
    baseline_models = {model: mse for model, mse in mse_data[dataset]['models'].items() if model != 'nft'}
    
    for i, mse in enumerate(nft_mse):
        # Calculate the percentage of the predictive range
        predictive_range_percentage = horizons[i] / max(horizons) * 100
        
        # Find the closest baseline MSE for the current NFT MSE
        closest_baseline_mse = min([mse_list[i] for mse_list in baseline_models.values()], key=lambda x: abs(x - mse))
        
        # Calculate the improvement percentage over the closest baseline MSE
        improvement_percentage = (closest_baseline_mse - mse) / closest_baseline_mse * 100
        
        datasets.append(dataset)
        horizons_list.append(horizons[i])
        predictive_ranges.append(predictive_range_percentage)
        nft_mses.append(mse)
        closest_baseline_mses.append(closest_baseline_mse)
        improvements.append(improvement_percentage)

# Create a dataframe for the results
df_results = pd.DataFrame({
    'dataset': datasets,
    'horizon': horizons_list,
    'predictive_range_percentage': predictive_ranges,
    'nft_mse': nft_mses,
    'closest_baseline_mse': closest_baseline_mses,
    'improvement_percentage': improvements
})

# Calculate the correlation
correlation = df_results.groupby('dataset').apply(lambda x: x['predictive_range_percentage'].corr(x['improvement_percentage']))
print("Correlation between the percentage of improvement and the percentage of the predictive range")
print(correlation)

# Save the detailed results to an Excel file
df_results.to_excel("precentage_results.xlsx", index=False)
print("Results saved to results.xlsx")


Correlation between the percentage of improvement and the percentage of the predictive range
dataset
Air Quality          0.009012
Chorales            -0.968252
ECG                 -0.487680
ECG (600)            0.627888
EEG                 -0.834286
ETTh1               -0.627968
ETTh1_long          -0.115459
ETTh2                0.126978
ETTm1               -0.522537
ETTm2                0.591803
Electricity          0.089093
Exchange Rate all    0.441258
ILI                 -0.963125
Traffic              0.100118
Weather             -0.093566
dtype: float64
Results saved to results.xlsx


In [76]:
ETT_mse_data = {
    'ETTh1': {
        'horizons': [1, 10, 24, 48, 96, 192, 336, 720],
        'models': {
            'nft': [0.07, 0.14, 0.18, 0.25, 0.39, 0.43, 0.44, 0.46],
            'timesnet': [0.15, 0.29, 0.33, 0.37, 0.4, 0.44, 0.49, 0.52],
            'patchtst': [0.1, 0.25, 0.29, 0.34, 0.37, 0.41, 0.42, 0.45],
            'dlinear': [0.12, 0.28, 0.31, 0.35, 0.37, 0.41, 0.44, 0.47],
            'Autoformer': [0.17, 0.4, 0.4, 0.427, 0.43, 0.46, 0.49, 0.52],
            'tcn': [0.14, 0.29, 0.43, 0.46, 0.46, 0.54, 0.53, 0.58],
            'nbeats': [0.11, 0.29, 0.33, 0.47, 0.71, 0.78, 0.79, 0.84]
        }
    },
    'ETTh2': {
        'horizons': [1, 10, 24, 48, 96, 192, 336, 720],
        'models': {
            'nft': [0.06, 0.1, 0.13, 0.16, 0.19, 0.23, 0.26, 0.34],
            'timesnet': [0.1, 0.15, 0.2, 0.25, 0.25, 0.4, 0.45, 0.46],
            'patchtst': [0.06, 0.12, 0.17, 0.22, 0.274, 0.34, 0.33, 0.38],
            'dlinear': [0.11, 0.13, 0.18, 0.25, 0.289, 0.38, 0.45, 0.61],
            'Autoformer': [0.12, 0.21, 0.26, 0.32, 0.332, 0.43, 0.48, 0.45],
            'tcn': [0.06, 0.11, 0.15, 0.19, 0.24, 0.48, 0.54, 0.55],
            'nbeats': [0.14, 0.18, 0.24, 0.41, 0.5, 1.2, 1.35, 1.73]
        }
    },
    'ETTm1': {
        'horizons': [1, 10, 24, 48, 96, 192, 336, 720],
        'models': {
            'nft': [0.04, 0.07, 0.11, 0.14, 0.16, 0.24, 0.37, 0.43],
            'timesnet': [0.05, 0.13, 0.17, 0.24, 0.26, 0.37, 0.41, 0.48],
            'patchtst': [0.05, 0.12, 0.21, 0.28, 0.293, 0.33, 0.37, 0.42],
            'dlinear': [0.05, 0.14, 0.24, 0.31, 0.299, 0.34, 0.37, 0.43],
            'Autoformer': [0.09, 0.24, 0.4, 0.56, 0.51, 0.51, 0.51, 0.53],
            'tcn': [0.04, 0.09, 0.14, 0.19, 0.23, 0.47, 0.45, 0.54],
            'nbeats': [0.05, 0.13, 0.23, 0.3, 0.35, 0.5, 0.58, 0.79]
        }
    },
    'ETTm2': {
        'horizons': [1, 10, 24, 48, 96, 192, 336, 720],
        'models': {
            'nft': [0.03, 0.05, 0.07, 0.1, 0.13, 0.17, 0.22, 0.26],
            'timesnet': [0.04, 0.07, 0.11, 0.14, 0.16, 0.25, 0.32, 0.41],
            'patchtst': [0.03, 0.07, 0.1, 0.14, 0.166, 0.22, 0.27, 0.36],
            'dlinear': [0.03, 0.07, 0.11, 0.15, 0.167, 0.22, 0.28, 0.40],
            'Autoformer': [0.07, 0.1, 0.14, 0.165, 0.205, 0.28, 0.34, 0.41],
            'tcn': [0.03, 0.06, 0.09, 0.12, 0.16, 0.3, 0.34, 0.42],
            'nbeats': [0.04, 0.07, 0.14, 0.18, 0.26, 0.59, 0.81, 1.00]
        }
    }
}


In [77]:
results = Calculate_percentage_MSE_reduction(ETT_mse_data)
Calculate_average_percentage_MSE(results)

{'ETTh1': 15.141755193504064, 'ETTh1_long': -4.316895292505052, 'ETTh2': 15.392303452481471, 'ETTm1': 15.661642578118553, 'ETTm2': 17.916140572390567}


{'ETTh1': 15.141755193504064,
 'ETTh1_long': -4.316895292505052,
 'ETTh2': 15.392303452481471,
 'ETTm1': 15.661642578118553,
 'ETTm2': 17.916140572390567}

In [74]:
ETT_mse_data = {
    'ETTh1': {
        'horizons': [1, 10, 24, 48, 96, 192, 336, 720],
        'models': {
            'nft': [0.07, 0.14, 0.18, 0.25, 0.39, 0.43, 0.44, 0.46],
            'timesnet': [0.15, 0.29, 0.33, 0.37, 0.38, 0.44, 0.49, 0.52],
            'patchtst': [0.1, 0.25, 0.29, 0.34, 0.37, 0.41, 0.42, 0.45],
            'dlinear': [0.12, 0.28, 0.31, 0.35, 0.37, 0.41, 0.44, 0.47],
            'Autoformer': [0.17, 0.4, 0.4, 0.427, 0.44, 0.46, 0.49, 0.52],
            'tcn': [0.14, 0.29, 0.43, 0.46, 0.46, 0.54, 0.53, 0.58],
            'nbeats': [0.11, 0.29, 0.33, 0.47, 0.71, 0.78, 0.79, 0.84]
        }
    },
    'ETTh1_long': {
        'horizons': [96, 192, 336, 720],
        'models': {
            'nft': [0.39, 0.43, 0.44, 0.46],
            'timesnet': [0.4, 0.44, 0.49, 0.52],
            'patchtst': [0.37, 0.41, 0.42, 0.45],
            'dlinear': [0.37, 0.41, 0.44, 0.47],
            'Autoformer': [0.43, 0.46, 0.49, 0.52],
            'tcn': [0.46, 0.54, 0.53, 0.58],
            'nbeats': [0.71, 0.78, 0.79, 0.84]
        }
    },
    'ETTh2': {
        'horizons': [1, 10, 24, 48, 96, 192, 336, 720],
        'models': {
            'nft': [0.06, 0.1, 0.13, 0.16, 0.19, 0.23, 0.26, 0.34],
            'timesnet': [0.1, 0.15, 0.2, 0.25, 0.34, 0.4, 0.45, 0.46],
            'patchtst': [0.06, 0.12, 0.17, 0.22, 0.274, 0.34, 0.33, 0.38],
            'dlinear': [0.11, 0.13, 0.18, 0.25, 0.289, 0.38, 0.45, 0.61],
            'Autoformer': [0.12, 0.21, 0.26, 0.32, 0.332, 0.43, 0.48, 0.45],
            'tcn': [0.06, 0.11, 0.15, 0.19, 0.24, 0.48, 0.54, 0.55],
            'nbeats': [0.14, 0.18, 0.24, 0.41, 1.65, 1.95, 1.75, 1.73]
        }
    },
    'ETTm1': {
        'horizons': [1, 10, 24, 48, 96, 192, 336, 720],
        'models': {
            'nft': [0.04, 0.07, 0.11, 0.14, 0.16, 0.24, 0.37, 0.43],
            'timesnet': [0.05, 0.13, 0.20, 0.29, 0.34, 0.37, 0.41, 0.48],
            'patchtst': [0.05, 0.12, 0.21, 0.28, 0.293, 0.33, 0.37, 0.42],
            'dlinear': [0.05, 0.14, 0.24, 0.31, 0.299, 0.34, 0.37, 0.43],
            'Autoformer': [0.09, 0.24, 0.4, 0.56, 0.51, 0.51, 0.51, 0.53],
            'tcn': [0.04, 0.09, 0.14, 0.19, 0.18, 0.47, 0.45, 0.54],
            'nbeats': [0.05, 0.13, 0.23, 0.3, 0.35, 0.5, 0.58, 0.79]
        }
    },
    'ETTm2': {
        'horizons': [1, 10, 24, 48, 96, 192, 336, 720],
        'models': {
            'nft': [0.03, 0.05, 0.07, 0.1, 0.13, 0.17, 0.22, 0.26],
            'timesnet': [0.04, 0.07, 0.11, 0.15, 0.16, 0.25, 0.32, 0.41],
            'patchtst': [0.03, 0.07, 0.1, 0.14, 0.166, 0.22, 0.27, 0.36],
            'dlinear': [0.03, 0.07, 0.11, 0.15, 0.167, 0.22, 0.28, 0.40],
            'Autoformer': [0.07, 0.1, 0.14, 0.165, 0.205, 0.28, 0.34, 0.41],
            'tcn': [0.03, 0.06, 0.09, 0.12, 0.16, 0.3, 0.34, 0.42],
            'nbeats': [0.04, 0.07, 0.14, 0.18, 0.26, 0.59, 0.81, 1.00]
        }
    }
}


In [60]:
mse_data_long = {
'Traffic long': {
    'horizons': [32, 48],
    'models': {
        'nft': [0.26, 0.4],
        'timesnet': [0.62, 0.62],
        'patchtst': [0.63, 0.66],
        'dlinear': [0.67, 0.72],
        'Autoformer': [0.67, 0.65],
        'tcn': [0.49, 0.51]
    }
},
    'Weather long': {
        'horizons': [60, 90, 120, 180],
        'models': {
            'nft': [0.26, 0.27, 0.28, 0.27],
            'timesnet': [0.37, 0.37, 0.38, 0.37],
            'patchtst': [0.40, 0.42, 0.42, 0.43],
            'dlinear': [0.37, 0.40, 0.41, 0.40],
            'Autoformer': [0.42, 0.44, 0.46, 0.47],
            'tcn': [0.36, 0.33, 0.33, 0.35]
        }
    },

    'Electricity long': {
        'horizons': [32],
        'models': {
            'nft': [0.13],
            'timesnet': [0.17],
            'patchtst': [0.17],
            'dlinear': [0.18],
            'Autoformer': [0.60],
            'tcn': [0.44]
        }
    },
    'Exchange Rate long': {
    'horizons': [96, 192, 336, 720],
    'models': {
        'nft': [0.02, 0.03, 0.04, 0.05],
        'timesnet': [0.107, 0.226, 0.367, 0.964],
        'patchtst': [0.09, 0.18, 0.4, 0.94],
        'dlinear': [0.088, 0.176, 0.313, 0.839],
        'Autoformer': [0.197, 0.300, 0.509, 1.447],
        'tcn': [0.09, 0.17, 0.24, 0.41]
    }
    },
    'ILI long': {
        'horizons': [36, 48, 60],
        'models': {
            'nft': [0.31, 0.35, 0.42],
            'timesnet': [1.972, 2.238, 2.027],
            'patchtst': [1.579, 1.553, 1.470],
            'dlinear': [1.963, 2.13, 2.368],
            'Autoformer': [3.103, 2.669, 2.770],
            'tcn': [0.94, 1.01, 1.03]
        }
    },
    'Air Quality long': {
        'horizons': [20, 25, 30],
        'models': {
            'nft': [0.35, 0.38, 0.4],
            'timesnet': [0.85, 0.92, 0.96],
            'patchtst': [0.75, 0.78, 0.83],
            'dlinear': [0.87, 0.93, 1.01],
            'Autoformer': [0.98, 1.03, 0.99],
            'tcn': [0.69, 0.68, 0.69]
        }
    },
   'ECG (600) long': {
        'horizons': [20, 25, 30],
        'models': {
            'nft': [0.15, 0.15, 0.1],
            'timesnet': [0.15,0.15,0.1],
            'patchtst': [0.15 ,0.15 ,0.18],
            'dlinear': [0.14 ,0.16 ,0.19],
            'Autoformer': [0.46 ,0.44 ,0.45],
            'tcn': [0.16 ,0.18 ,0.21]
        }
    },
   'ECG long': {
        'horizons': [50, 100],
        'models': {
            'nft': [0.14, 0.18],
            'timesnet': [1.16 ,1.09],
            'patchtst': [0.94 ,0.98],
            'dlinear': [0.66 ,0.69],
            'Autoformer': [1.15 ,1.26],
            'tcn': [0.70 ,0.81]
        }
    },
    'EEG long': {
        'horizons': [25],
        'models': {
            'nft': [0.16],
            'timesnet': [0.44],
            'patchtst': [0.42],
            'dlinear': [0.435],
            'Autoformer': [0.62],
            'tcn': [0.39]
        }
    },
    'Chorales long': {
        'horizons': [4, 5],
        'models': {
            'nft': [0.23, 0.22],
            'timesnet': [0.31, 0.32],
            'patchtst': [0.30, 0.31],
            'dlinear': [0.37, 0.36],
            'Autoformer': [0.26, 0.24],
            'tcn': [0.37, 0.56]
        }
    },
    'ETTh1 long': {
        'horizons': [48, 96],
        'models': {
            'nft': [0.25, 0.39],
            'timesnet': [0.37, 0.4],
            'patchtst': [0.34, 0.37],
            'dlinear': [0.35, 0.37],
            'Autoformer': [0.427, 0.43],
            'tcn': [0.46, 0.46],
            'nbeats': [0.47, 0.71]
        }
    },
    'ETTh2 long': {
        'horizons': [48, 96],
        'models': {
            'nft': [0.16, 0.19],
            'timesnet': [0.25, 0.25],
            'patchtst': [0.22, 0.274],
            'dlinear': [0.25, 0.289],
            'Autoformer': [0.32, 0.332],
            'tcn': [0.19, 0.24],
            'nbeats': [0.41, 0.5]
        }
    },
   'ETTm1 long': {
        'horizons': [48, 96],
        'models': {
            'nft': [0.14, 0.16],
            'timesnet': [0.24, 0.26],
            'patchtst': [0.28, 0.293],
            'dlinear': [0.31, 0.299],
            'Autoformer': [0.56, 0.51],
            'tcn': [0.19, 0.23],
            'nbeats': [0.3, 0.35]
        }
    },
    'ETTm2 long': {
        'horizons': [48, 96],
        'models': {
            'nft': [0.1, 0.13],
            'timesnet': [0.14, 0.16],
            'patchtst': [0.14, 0.166],
            'dlinear': [0.15, 0.167],
            'Autoformer': [0.165, 0.205],
            'tcn': [0.12, 0.16],
            'nbeats': [0.18, 0.26]
        }
    }
    }

In [61]:
mse_data_sort = {
'Traffic sort': {
    'horizons': [1, 16],
    'models': {
        'nft': [0.09, 0.18],
        'timesnet': [0.53, 0.58],
        'patchtst': [0.26, 0.58],
        'dlinear': [0.4, 0.63],
        'Autoformer': [0.5, 0.56],
        'tcn': [0.1, 0.46]
    }
},
'Weather sort': {
        'horizons': [1, 7, 15, 30],
        'models': {
            'nft': [0.19, 0.24, 0.24, 0.25],
            'timesnet': [0.33, 0.35, 0.44, 0.36],
            'patchtst': [0.34, 0.36, 0.37, 0.38],
            'dlinear': [0.29, 0.32, 0.33, 0.35],
            'Autoformer': [0.37, 0.37, 0.37, 0.38],
            'tcn': [0.25, 0.29, 0.30, 0.32]
        }
    },

'Electricity sort': {
        'horizons': [1, 16],
        'models': {
            'nft': [0.04, 0.08],
            'timesnet': [0.14, 0.16],
            'patchtst': [0.09, 0.20],
            'dlinear': [0.07, 0.16],
            'Autoformer': [0.20, 0.40],
            'tcn': [0.05, 0.32]
        }
    },
'Exchange Rate sort': {
        'horizons': [1, 16, 32, 48],
        'models': {
            'nft': [0.0001, 0.01, 0.02, 0.02],
            'timesnet': [0.05, 0.18, 0.15, 0.11],
            'patchtst': [0.01, 0.02, 0.03, 0.04],
            'dlinear': [0.01, 0.02, 0.03, 0.05],
            'Autoformer': [0.04, 0.04, 0.06, 0.09],
            'tcn': [0.2, 0.11, 0.04, 0.06]
        }
    },
'ILI sort': {
        'horizons': [1, 12, 24],
        'models': {
            'nft': [0.05, 0.18, 0.25],
            'timesnet': [0.64, 1.01, 2.317],
            'patchtst': [0.24, 1.25, 1.319],
            'dlinear': [2.32, 2.83, 2.215],
            'Autoformer': [0.84, 2.47, 3.483],
            'tcn': [0.86, 0.98, 0.95]
        }
    },
'Air Quality sort': {
        'horizons': [1, 5, 10, 15],
        'models': {
            'nft': [0.13, 0.21, 0.28, 0.31],
            'timesnet': [0.74, 0.81, 0.85, 0.87],
            'patchtst': [0.22, 0.58, 0.68, 0.72],
            'dlinear': [0.39, 0.69, 0.78, 0.83],
            'Autoformer': [0.72, 0.86, 0.97, 1.02],
            'tcn': [0.21, 0.42, 0.53, 0.64]
        }
    },
'ECG (600) sort': {
        'horizons': [1, 5, 10, 15],
        'models': {
            'nft': [0.02, 0.08, 0.07, 0.1],
            'timesnet': [0.02,0.08,0.07,0.1],
            'patchtst': [0.02 ,0.05 ,0.08 ,0.1],
            'dlinear': [0.02 ,0.06 ,0.08 ,0.11],
            'Autoformer': [0.8,0.49,0.38, 0.41],
            'tcn': [ 0.03 ,0.07 ,0.1 ,0.12]
        }
    },
'ECG sort': {
        'horizons': [1, 10, 25],
        'models': {
            'nft': [0.04, 0.08, 0.12],
            'timesnet': [0.31 ,0.49 ,0.625],
            'patchtst': [0.34 ,0.57 ,0.77],
            'dlinear': [0.35 ,0.46 ,0.58],
            'Autoformer': [1.62 ,1.13 ,1.5],
            'tcn': [0.16 ,0.42 ,0.56]
        }
    },
'EEG sort': {
        'horizons': [1, 10],
        'models': {
            'nft': [0.03, 0.1],
            'timesnet': [0.16, 0.35],
            'patchtst': [0.15, 0.345],
            'dlinear': [0.135, 0.365],
            'Autoformer': [0.63, 0.545],
            'tcn': [0.09, 0.32],
            'tcn': [0.09, 0.32],
        }
    },
'Chorales sort': {
        'horizons': [1, 2, 3],
        'models': {
            'nft': [0.15, 0.18, 0.20],
            'timesnet': [0.29, 0.3, 0.31],
            'patchtst': [0.34, 0.27, 0.29],
            'dlinear': [0.29, 0.29, 0.40],
            'Autoformer': [0.30, 0.25, 0.25],
            'tcn': [0.27, 0.63, 0.62]
        }
    },
 'ETTh1 sort': {
        'horizons': [1, 10, 24],
        'models': {
            'nft': [0.07, 0.14, 0.18],
            'timesnet': [0.15, 0.29, 0.33],
            'patchtst': [0.1, 0.25, 0.29],
            'dlinear': [0.12, 0.28, 0.31],
            'Autoformer': [0.17, 0.4, 0.4],
            'tcn': [0.14, 0.29, 0.43],
            'nbeats': [0.11, 0.29, 0.33]
        }
    },
'ETTh2 short': {
        'horizons': [1, 10, 24],
        'models': {
            'nft': [0.06, 0.1, 0.13],
            'timesnet': [0.1, 0.15, 0.2],
            'patchtst': [0.06, 0.12, 0.17],
            'dlinear': [0.11, 0.13, 0.18],
            'Autoformer': [0.12, 0.21, 0.26],
            'tcn': [0.06, 0.11, 0.15],
            'nbeats': [0.14, 0.18, 0.24]
        }
    },
'ETTm1 short': {
        'horizons': [1, 10, 24],
        'models': {
            'nft': [0.04, 0.07, 0.11],
            'timesnet': [0.05, 0.13, 0.17],
            'patchtst': [0.05, 0.12, 0.21],
            'dlinear': [0.05, 0.14, 0.24],
            'Autoformer': [0.09, 0.24, 0.4],
            'tcn': [0.04, 0.09, 0.14],
            'nbeats': [0.05, 0.13, 0.23]
        }
    },
'ETTm2 short': {
        'horizons': [1, 10, 24],
        'models': {
            'nft': [0.03, 0.05, 0.07],
            'timesnet': [0.04, 0.07, 0.11],
            'patchtst': [0.03, 0.07, 0.1],
            'dlinear': [0.03, 0.07, 0.11],
            'Autoformer': [0.07, 0.1, 0.14],
            'tcn': [0.03, 0.06, 0.09],
            'nbeats': [0.04, 0.07, 0.14]
        }
    },
    }


In [75]:
results = Calculate_percentage_MSE_reduction(ETT_mse_data)
Calculate_average_percentage_MSE(results)

{'ETTh1': 15.141755193504064, 'ETTh1_long': -4.316895292505052, 'ETTh2': 15.392303452481471, 'ETTm1': 13.246183640920481, 'ETTm2': 17.916140572390567}


{'ETTh1': 15.141755193504064,
 'ETTh1_long': -4.316895292505052,
 'ETTh2': 15.392303452481471,
 'ETTm1': 13.246183640920481,
 'ETTm2': 17.916140572390567}

In [68]:
results = Calculate_percentage_MSE_reduction(mse_data_long)
Calculate_average_percentage_MSE(results)

{'Traffic long': 34.25370148059224, 'Weather long': 20.992063492063483, 'Electricity long': 23.529411764705888, 'Exchange Rate long': 82.69096995782792, 'ILI long': 63.86370407336127, 'Air Quality long': 45.14066496163682, 'ECG (600) long': -2.38095238095238, 'ECG long': 76.35046113306981, 'EEG long': 58.97435897435898, 'Chorales long': 9.935897435897434, 'ETTh1 long': 10.532591414944353, 'ETTh2 long': 18.311403508771928, 'ETTm1 long': 28.37528604118993, 'ETTm2 long': 17.708333333333332}


{'Traffic long': 34.25370148059224,
 'Weather long': 20.992063492063483,
 'Electricity long': 23.529411764705888,
 'Exchange Rate long': 82.69096995782792,
 'ILI long': 63.86370407336127,
 'Air Quality long': 45.14066496163682,
 'ECG (600) long': -2.38095238095238,
 'ECG long': 76.35046113306981,
 'EEG long': 58.97435897435898,
 'Chorales long': 9.935897435897434,
 'ETTh1 long': 10.532591414944353,
 'ETTh2 long': 18.311403508771928,
 'ETTm1 long': 28.37528604118993,
 'ETTm2 long': 17.708333333333332}

In [43]:

# Example usage
t_stat_dict, p_value_dict = Calulate_t_test(mse_data)  # Assume 'data' is your dataset dictionary
avg_t_stat, avg_p_value, overall_avg_t_stat, overall_avg_p_value = calculate_average_t_p(t_stat_dict, p_value_dict)
print(avg_t_stat)
print(avg_p_value)
print(overall_avg_t_stat)
print(overall_avg_p_value)


{'Traffic': -7.214183059240673, 'Weather': -16.726630524151748, 'Electricity': -3.241551415516599, 'Exchange Rate all': -2.2912520668139242, 'ILI': -10.540739268437614, 'Air Quality': -20.98816380096026, 'ECG (600)': nan, 'ECG': -7.423951836497407, 'EEG': -5.3299385857332044, 'Chorales': -6.579759013598023, 'ETTh1': -3.503439264787982, 'ETTh2': -4.647860536200922, 'ETTm1': -3.5151120652345456, 'ETTm2': -4.476851942026864}
{'Traffic': 0.02017157159642987, 'Weather': 6.837687323558146e-06, 'Electricity': 0.09514893737689076, 'Exchange Rate all': 0.07940561637215528, 'ILI': 0.0009555415794387191, 'Air Quality': 0.00010325431219769861, 'ECG (600)': nan, 'ECG': 0.005275456507865628, 'EEG': 0.048694868942424106, 'Chorales': 0.01451445575628385, 'ETTh1': 0.031300699082367656, 'ETTh2': 0.025166177706753067, 'ETTm1': 0.02661938282105022, 'ETTm2': 0.02398087791332816}
-7.421494875323059
0.028564898281116042


In [19]:

results_ablation_lstm, results_ablation_fc = Calculate_percentage_MSE_reduction_ablation(data_ablation_mse)
average_mse_lstm_results = Calculate_average_percentage_MSE(results_ablation_lstm)
average_mse_fc_results = Calculate_average_percentage_MSE(results_ablation_fc)
average_dict_values(average_mse_lstm_results)
average_dict_values(average_mse_fc_results)


{'Weather': 33.37132792820345, 'Air Quality': 77.25121721285042, 'ECG (600)': 26.39455782312925, 'ECG': 69.20967032643395, 'EEG': 74.36686965720331, 'Chorales': 26.376106295360955}
{'Weather': 38.08044090056285, 'Air Quality': 77.96078248167187, 'ECG (600)': 37.91011619958988, 'ECG': 72.88971220321335, 'EEG': 74.3709300233855, 'Chorales': 33.15270935960591}
51.16162487386356
55.72744852800489


55.72744852800489